In [ ]:

import sys
sys.path.append('../src')

from data import GANCifar10Data
from models import build_dcgan_generator, build_dcgan_discriminator, build_dcgan_gan
from train import DCGANTrainer
from utils import plot_gan_training_history, save_gan_results, generate_final_gan_samples, calculate_fid, update_summary_csv
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

# Configuración

NOTEBOOK = "2_dcgan_generation"
MODEL_NAME = "DCGAN(CIFAR10-64)"
LATENT_DIM = 100


print("=" * 60)
print("PROYECTO 3: RAG PARA QUESTION ANSWERING")
print("=" * 60)


PROYECTO 2: GENERACIÓN DE IMÁGENES CON DCGAN
Dataset: CIFAR-10


In [1]:
# 1. CONFIGURACIÓN INICIAL
print("🎭 EJERCICIO 2: DCGAN CON CELEBA - GENERACIÓN DE CARAS")

import tensorflow as tf
import sys
import os
sys.path.append('../src')

from data import CelebAData
from model import build_dcgan_generator, build_dcgan_discriminator, build_dcgan_gan
from train import DCGANTrainer
from utils import plot_gan_training_history, generate_final_gan_samples, save_gan_results


🎭 EJERCICIO 2: DCGAN CON CELEBA - GENERACIÓN DE CARAS


ModuleNotFoundError: No module named 'model'